# Speeding up simulations with JIT-compilation and GPUs

In this tutorial, you will learn how to:  
- set parameters of `Jaxley` models such as compartment radius or channel conductances
- set initial states
- set synaptic parameters

Here is a code snippet which you will learn to understand in this tutorial:
```python
cell = ...  # See tutorial on Basics of Jaxley.
cell.insert(Na())

cell.set("radius", 1.0)  # Set compartment radius.
cell.branch(0).set("Na_gNa", 0.1)  # Set sodium maximal conductance.
cell.set("v", -65.0)  # Set initial voltage.

net = ...  # See tutorial on Networks of Jaxley.
fully_connect(net.cell(0), net.cell(1), IonotropicSynapse())
net.IonotropicSynapse().set("IonotropicSynapse_gS", 0.01)
```

In the previous two tutorials, you learned how to build single cells or networks and how to simulate them. In this tutorial, you will learn how to speed up such simulations by many orders of magnitude. This can be achieved in to ways:
- by using JIT compilation  
- by using GPU parallelization

Let's get started!

In [32]:
import matplotlib.pyplot as plt
import numpy as np
import jax.numpy as jnp
from jax import jit, vmap

import jaxley as jx
from jaxley.channels import Na, K, Leak

### Preface: Building the cell or network

We first build a cell (or network) in the same way as we showed in the previous tutorials:

In [33]:
dt = 0.025
t_max = 10.0

comp = jx.Compartment()
branch = jx.Branch(comp, nseg=2)
cell = jx.Cell(branch, parents=[-1, 0])

### Setting parameters in `Jaxley`

To modify parameters of the simulation, you can use the `.set()` method. For example
```python
cell.set("radius", 0.1)
```
will modify the radius of every compartment in the cell to 0.1 micrometer. You can also modify the parameters only of some branches:
```python
cell.branch(0).set("radius", 1.0)
```
or even of compartments:
```python
cell.branch(0).comp(0).set("radius", 10.0)
```

You can always inspect the current parameters by inspecting `cell.nodes`, which is a pandas Dataframe that contains all information about the cell. You can use `.set()` to set morphological parameters, channel parameters, synaptic parameters, and initial states, as outlined below:

### Setting morphological parameters

`Jaxley` allows to set the following morphological parameters:
- `radius`: the radius of the (zylindrical) compartment (in micrometer)
- `length`: the length of the zylindrical compartment (in micrometer)
- `axial_resistivity`: the resistivity of current flow between compartments (in ohm centimeter)

In [34]:
cell.branch(0).set("axial_resistivity", 1000.0)
cell.set("length", 1.0)  # This will set every compartment in the cell to have length 1.0.

In [35]:
cell.nodes

,comp_index,branch_index,cell_index,length,radius,axial_resistivity,capacitance,v
0,0,0,0,1.0,1.0,1000.0,1.0,-70.0
1,1,0,0,1.0,1.0,1000.0,1.0,-70.0
2,2,1,0,1.0,1.0,5000.0,1.0,-70.0
3,3,1,0,1.0,1.0,5000.0,1.0,-70.0


### Setting channel parameters

You can also modify channel parameters. Every parameter that should be modifiable has to be defined in `self.channel_params` of the channel.

In [36]:
cell.insert(Na())
cell.branch(1).comp(0).set("Na_gNa", 0.1)

In [37]:
cell.nodes

,comp_index,branch_index,cell_index,length,radius,axial_resistivity,capacitance,v,Na,Na_gNa,eNa,vt,Na_m,Na_h
0,0,0,0,1.0,1.0,1000.0,1.0,-70.0,True,0.05,50.0,-60.0,0.2,0.2
1,1,0,0,1.0,1.0,1000.0,1.0,-70.0,True,0.05,50.0,-60.0,0.2,0.2
2,2,1,0,1.0,1.0,5000.0,1.0,-70.0,True,0.10,50.0,-60.0,0.2,0.2
3,3,1,0,1.0,1.0,5000.0,1.0,-70.0,True,0.05,50.0,-60.0,0.2,0.2


### Setting synaptic parameters

In order to set parameters of synapses, you have to use `net.SynapseName.set()`, e.g.:

In [38]:
from jaxley.synapses import IonotropicSynapse
from jaxley.connect import fully_connect

num_cells = 2
net = jx.Network([cell for _ in range(num_cells)])
fully_connect(net.cell(0), net.cell(1), IonotropicSynapse())

# Unlike for channels, you have to index into the synapse with `net.SynapseName`
net.IonotropicSynapse.set("IonotropicSynapse_gS", 0.1)

/Users/michaeldeistler/Documents/phd/jaxley/jaxley/modules/base.py:1533: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self.pointer.edges = pd.concat(


You can inspect synaptic parameters and states with `net.edges`:

In [39]:
net.edges

,pre_locs,pre_branch_index,pre_cell_index,post_locs,post_branch_index,post_cell_index,type,type_ind,global_pre_comp_index,global_post_comp_index,global_pre_branch_index,global_post_branch_index,IonotropicSynapse_gS,IonotropicSynapse_e_syn,IonotropicSynapse_k_minus,IonotropicSynapse_s
0,0.25,0,0,0.25,1,1,IonotropicSynapse,0,0,6,0,3,0.1,0.0,0.025,0.2


### Setting initial states

Finally, you can also set initial states. These include the initial voltage `v` and the states of all channels and synapses (which must be defined in `self.channel_states` of the channel. For example:

In [40]:
net.set("v", -72.0)
net.IonotropicSynapse.set("IonotropicSynapse_s", 0.1)

In [41]:
net.nodes

,comp_index,branch_index,cell_index,length,radius,axial_resistivity,capacitance,v,Na,Na_gNa,eNa,vt,Na_m,Na_h
0,0,0,0,1.0,1.0,1000.0,1.0,-72.0,True,0.05,50.0,-60.0,0.2,0.2
1,1,0,0,1.0,1.0,1000.0,1.0,-72.0,True,0.05,50.0,-60.0,0.2,0.2
2,2,1,0,1.0,1.0,5000.0,1.0,-72.0,True,0.10,50.0,-60.0,0.2,0.2
3,3,1,0,1.0,1.0,5000.0,1.0,-72.0,True,0.05,50.0,-60.0,0.2,0.2
4,4,2,1,1.0,1.0,1000.0,1.0,-72.0,True,0.05,50.0,-60.0,0.2,0.2
5,5,2,1,1.0,1.0,1000.0,1.0,-72.0,True,0.05,50.0,-60.0,0.2,0.2
6,6,3,1,1.0,1.0,5000.0,1.0,-72.0,True,0.10,50.0,-60.0,0.2,0.2
7,7,3,1,1.0,1.0,5000.0,1.0,-72.0,True,0.05,50.0,-60.0,0.2,0.2


In [42]:
net.edges

,pre_locs,pre_branch_index,pre_cell_index,post_locs,post_branch_index,post_cell_index,type,type_ind,global_pre_comp_index,global_post_comp_index,global_pre_branch_index,global_post_branch_index,IonotropicSynapse_gS,IonotropicSynapse_e_syn,IonotropicSynapse_k_minus,IonotropicSynapse_s
0,0.25,0,0,0.25,1,1,IonotropicSynapse,0,0,6,0,3,0.1,0.0,0.025,0.1


### Summary

You can now modify parameters of your `Jaxley` simulation. In [the next tutorial](), you will learn how to make parameter sweeps (or stimulus sweeps) fast with jit-compilation and GPU parallelization.